<H1>AML Project 2: Decision Trees, SVMs and Ensemble Learning</H1>
<p1>Andrew J Markland, ajm259@uakron.edu</p1> <br>
<p1>The University of Akron: CEPS : School of Computer Science</p1><br>
<p1>Applied Machine Learning: CPSC:436:010</p1><br>
<p1>Dr. Zhong-Hui Duan </p1><br><br>
<p1>Abstract: The purpose of this assingment is to display understanding and untilization of Decesion tree, ensemble learning</p1><br>
<p1>and SVM's through predicting the likelyhood of strokes through models that we will train with data retrived from NHANES</p1><br><br>
<p1>Disclaimer** While this project is public in my public repository on github, I do not discourage others from using it as a </p1><br>
<p1>reference for their own implementations, but the work written here is expressly written and owned by Andrew J Markland.</p1><br>
<p1>I do not condone nor tolerate academic dishonesty, and should the question of plagerism arise derived from my works</p1><br>
<p1>copies of this repository shall be promptly submitted to the assigning professor and the department of student affairs.</p1>

In [2]:
#includes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [387]:
"""
    Data Manager
    This cell will handle partitioning and cleaning the data and provide a hand-full of useful
    variables that I can call on to test different data sets based on a partition of the test data, this is primarily 
    going to be used to streamline the testing phases and make data balancing and acquistion faster. 
"""
class DataWrapper:
    def __init__(self):
        self.train_x = None
        self.train_y = None
        self.test_x = None
        self.test_y = None

class finalWrapper:
    def __init__(self):
        self.train_x = None
        self.train_y = None

initialDataPD = pd.read_csv("NHANES_data_stroke_train.csv")
test_DF = pd.read_csv("NHANES_data_stroke_test4Students.csv")
test_DF = test_DF.drop("stroke", axis=1)

def partitionAndBalance(dataObject):
    global initialDataPD
    X = initialDataPD.drop('stroke', axis=1)
    y = initialDataPD['stroke']
    dataObject.train_x, dataObject.test_x, dataObject.train_y, dataObject.test_y = train_test_split(X, y, test_size = 0.2)
    balanceTrainData(dataObject)
    imputeMissingTestValues(dataObject)
    #print(dataObject.train_X)
    return dataObject
    
    
def balanceTrainData(dataObject):
    combinedDataDF = pd.DataFrame(dataObject.train_x)
    combinedDataDF['stroke'] = dataObject.train_y
    """partition based on classes"""
    strokeDF = combinedDataDF[combinedDataDF['stroke'] == 1]
    noStrokeDF = combinedDataDF[combinedDataDF['stroke'] == 2]
    """clean up missing values"""
    noStrokeDF = noStrokeDF.dropna()
    imputer = KNNImputer(n_neighbors=3)
    imputedDF = imputer.fit_transform(strokeDF)
    cleanStrokeDF = pd.DataFrame(imputedDF, columns=strokeDF.columns)
    noStrokeDF = noStrokeDF.sample(n=len(cleanStrokeDF))
    balancedDF = pd.concat([noStrokeDF, cleanStrokeDF])
    dataObject.train_x = balancedDF.drop('stroke', axis=1)
    dataObject.train_y = balancedDF['stroke']
    
def imputeMissingTestValues(dataObject):
    imputer = KNNImputer(n_neighbors=3)
    imputedTestDF = imputer.fit_transform(dataObject.test_x)
    dataObject.test_x = pd.DataFrame(imputedTestDF, columns=dataObject.test_x.columns)
    
def dropRows(dataObject, rows):
    for row in rows:
        dataObject.test_x.drop(row, axis=1)
        dataObject.train_x.drop(row, axis=1)
        
def scaleData(dataObject):
    scaler = StandardScaler()
    trainScaled = scaler.fit_transform(dataObject.train_x)
    testScaled = scaler.transform(dataObject.test_x)
    dataObject.test_x = pd.DataFrame(testScaled, columns=dataObject.test_x.columns)
    dataObject.train_x = pd.DataFrame(trainScaled, columns=dataObject.train_x.columns)

def imputeMissingValuesDataFrame(data):
    imputer = KNNImputer(n_neighbors = 3)
    cleanData = imputer.fit_transform(data)
    data = pd.DataFrame(cleanData, columns=data.columns)
    return data

def dropRowsDataframe(data, rows):
    for row in rows:
        data.drop(row, axis=1)
    return data

def cutParticipantID(data):
    patientIds = pd.DataFrame(data["ParticipantID"], columns=["ParticipantID"])
    data.drop("ParticipantID", axis=1);
    return patientIds
    
def balanceFinalWrapper(Fwrapper):
    #load data and separate
    strokeDF = initialDataPD[initialDataPD['stroke'] == 1]
    noStrokeDF = initialDataPD[initialDataPD['stroke'] == 2]
    #clean the data
    noStrokeDF = noStrokeDF.dropna()
    imputer = KNNImputer(n_neighbors = 3)
    cleanStroke = imputer.fit_transform(strokeDF)
    strokeDF = pd.DataFrame(cleanStroke, columns=strokeDF.columns)
    noStrokeDF = noStrokeDF.sample(n=len(strokeDF))
    #join the data back together
    FTrainDF = pd.concat([strokeDF, noStrokeDF])
    #set the training data 
    Fwrapper.train_x = FTrainDF.drop("stroke", axis=1)
    Fwrapper.train_y = FTrainDF["stroke"]
    

In [370]:
"""
Random Forest run,
    This cell is the testing of the random forest model
"""


#getting a random distribution of data and balancing it
rf_dataObject = DataWrapper()
rf_dataObject = partitionAndBalance(rf_dataObject)

noise = ["Income", "Sex", "Race", "Edu", "Diabetes", "isInsured"]
dropRows(rf_dataObject, noise)

# initializing the random forest
randForest = RandomForestClassifier(n_estimators = 1000, max_depth = 12, min_samples_split=20, min_samples_leaf=15, max_features=5)

randForest.fit(rf_dataObject.train_x, rf_dataObject.train_y)

prediction = randForest.predict(rf_dataObject.test_x)
accuracy = accuracy_score(rf_dataObject.test_y, prediction)
print(accuracy)

def getForestRun(accuracyList):
    #getting a random distribution of data and balancing it 
    rf_dataObject = DataWrapper()
    rf_dataObject = partitionAndBalance(rf_dataObject)
    
    noise = ["Income", "Sex", "Race", "Edu", "Diabetes", "isInsured"]
    dropRows(rf_dataObject, noise)
    rf_dataObject.train_x.drop("ParticipantID", axis=1)
    rf_dataObject.test_x.drop("ParticipantID", axis=1)

    # initializing the random forest
    randForest = RandomForestClassifier(n_estimators = 1000, max_depth = 12, min_samples_split=20, min_samples_leaf=15, max_features=5)

    randForest.fit(rf_dataObject.train_x, rf_dataObject.train_y)

    prediction = randForest.predict(rf_dataObject.test_x)
    accuracy = accuracy_score(rf_dataObject.test_y, prediction)
    accuracyList.append(accuracy)

0.684981684981685


In [357]:
"""
 Gradient Boosted Tree
    this cell is for testing the gradient boosted tree model
"""

#getting a Random Distribution of data and balancing it
grad_dataObject = partitionAndBalance(DataWrapper())

noise = ["Income", "Sex", "Race", "Edu", "Diabetes", "BMI"]
dropRows(grad_dataObject, noise)

grad_dataObject.test_x.drop("ParticipantID", axis=1)
grad_dataObject.train_x.drop("ParticipantID", axis=1)

gradTree = GradientBoostingClassifier(n_estimators=500, max_depth=10, min_samples_split=20, learning_rate=0.1, min_samples_leaf=15, max_features=5)
gradTree.fit(grad_dataObject.train_x, grad_dataObject.train_y)

prediction = gradTree.predict(grad_dataObject.test_x)
accuracy = accuracy_score(grad_dataObject.test_y, prediction)
print(accuracy)

def getGradRun(accuracyList):
    #getting a Random Distribution of data and balancing it
    grad_dataObject = partitionAndBalance(DataWrapper())
    #noise = ["Income", "Sex", "Race", "Edu", "Diabetes"]
    dropRows(grad_dataObject, noise)
    grad_dataObject.test_x.drop("ParticipantID", axis=1)
    grad_dataObject.train_x.drop("ParticipantID", axis=1)

    gradTree = GradientBoostingClassifier(n_estimators=500, max_depth=10, min_samples_split=20, learning_rate=0.1, min_samples_leaf=15, max_features=5)
    gradTree.fit(grad_dataObject.train_x, grad_dataObject.train_y)

    prediction = gradTree.predict(grad_dataObject.test_x)
    accuracy = accuracy_score(grad_dataObject.test_y, prediction)
    accuracyList.append(accuracy)


0.6532356532356532


In [302]:
"""
Support Vector Machine
    This cell is for testing the support Vector Machine model
"""

svm_data = partitionAndBalance(DataWrapper())
SVM = SVC(kernel = 'rbf', C=.6, gamma=0.001)
scaleData(svm_data)
SVM.fit(svm_data.train_x, svm_data.train_y)

prediction = SVM.predict(svm_data.test_x)

accuracy = accuracy_score(svm_data.test_y, prediction)
print(accuracy)

def getSVMrun(accuracyList):
    svm_data = partitionAndBalance(DataWrapper())
    SVM = SVC(kernel = 'rbf', C=1, gamma=(1/18))
    svm_data.train_x.drop("ParticipantID", axis=1)
    svm_data.test_x.drop("ParticipantID", axis=1)
    scaleData(svm_data)
    SVM.fit(svm_data.train_x, svm_data.train_y)

    prediction = SVM.predict(svm_data.test_x)

    accuracy.append(accuracy_score(svm_data.test_y, prediction))

0.6874236874236874


In [388]:
"""
Testing cell
  this cell is mostly just to run 100 runs of each model with random partitions of
  the test data to see average performance across the runs
"""

accuracy = []
for x in range(100):
    getForestRun(accuracy)

print(sum(accuracy)/len(accuracy))
print(min(accuracy))
print(max(accuracy))

0.6886446886446884
0.6166056166056166
0.7509157509157509


In [384]:
"""
   Random Forests Final Run
    ***Discrete Values***
"""


train_x
      ParticipantID  Income  Sex   Age  Race       Edu  Diastolic    Systolic  \
0            1879.0    0.87  2.0  62.0   4.0  3.333333  92.333333  168.000000   
1            1905.0    1.52  2.0  69.0   3.0  2.333333  99.000000  130.666667   
2            1908.0    1.52  1.0  52.0   4.0  3.333333  87.666667  127.666667   
3            1921.0    0.06  2.0  64.0   4.0  2.333333  65.666667  110.333333   
4            1948.0    1.00  2.0  62.0   3.0  3.000000  75.333333  158.666667   
...             ...     ...  ...   ...   ...       ...        ...         ...   
3596         4601.0    0.56  2.0  40.0   1.0  1.000000  58.000000  110.666667   
3582         4587.0    0.02  2.0  35.0   1.0  3.000000  63.333333  106.000000   
191          1192.0    0.40  1.0  49.0   6.0  4.000000  77.333333  112.666667   
3947         4952.0    1.74  2.0  32.0   1.0  4.000000  75.333333  108.000000   
3528         4533.0    1.36  1.0  45.0   7.0  2.000000  72.000000  110.666667   

          Pulse    